LightFM

In [1]:
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
import numpy as np
import pandas as pd
from lightfm.cross_validation import random_train_test_split


/opt/anaconda3/envs/backup/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data = pd.read_csv("shopping_behavior_updated.csv")
data['Customer ID'] = data['Customer ID'].astype(str)

In [4]:
# Create age group
data['Age Group'] = pd.cut(data['Age'], bins=[0, 25, 35, 50, 100], labels=['<25', '25-35', '35-50', '>50'])
# Review Rating -> Postive or Negative
data['weighted_liked'] = data['Review Rating'].apply(lambda x: 3.0 if x >= 4.0 else 0.5)

In [5]:
#Create feature for user and item
user_features = [f"Age Group:{group}" for group in data['Age Group'].unique()] + \
                [f"Gender:{gender}" for gender in data['Gender'].unique()] + \
                [f"Previous Purchases:{purchases}" for purchases in data['Previous Purchases'].unique()]  # Thêm Previous Purchases

item_features = list(data['Category'].unique()) + \
                list(data['Season'].unique()) + \
                list(data['Color'].unique()) + \
                [f"Discount:{disc}" for disc in data['Discount Applied'].unique()] 


In [ ]:
# Dataset
dataset = Dataset()
dataset.fit(
    data['Customer ID'], 
    data['Item Purchased'], 
    user_features=user_features, 
    item_features=item_features
)

In [10]:
(interactions_matrix, _) = dataset.build_interactions(
    [(x['Customer ID'], x['Item Purchased'], x['weighted_liked']) for _, x in data.iterrows()]
)

user_features_matrix = dataset.build_user_features(
    [(x['Customer ID'], [f"Age Group:{x['Age Group']}", f"Gender:{x['Gender']}", 
                         f"Previous Purchases:{x['Previous Purchases']}"])  # Thêm Previous Purchases
     for _, x in data.iterrows()]
)

item_features_matrix = dataset.build_item_features(
    [(x['Item Purchased'], [x['Category'], x['Season'], x['Color'], f"Discount:{x['Discount Applied']}"]) 
     for _, x in data.iterrows()]
)

In [11]:
train_interactions, test_interactions = random_train_test_split(interactions_matrix, test_percentage=0.1, random_state=np.random.RandomState(42))

In [12]:
best_auc = 0
best_config = None
for loss in ['warp', 'bpr', 'logistic', 'warp-kos']:
    for components in [50, 100, 200, 300]:
        for epochs in [10, 30, 50]:
            # Khởi tạo mô hình
            model = LightFM(loss=loss, no_components=components, random_state=42)
            
            # Huấn luyện
            model.fit(train_interactions, user_features=user_features_matrix, 
                      item_features=item_features_matrix, epochs=epochs, num_threads=4)
            
            # Tính AUC
            test_auc = auc_score(model, interactions_matrix, 
                                 user_features=user_features_matrix, 
                                 item_features=item_features_matrix, 
                                 num_threads=4).mean()
            
            print(f"Loss: {loss}, Components: {components}, Epochs: {epochs}, AUC: {test_auc:.4f}")
            
            # Cập nhật cấu hình tốt nhất
            if test_auc > best_auc:
                best_auc = test_auc
                best_config = (loss, components, epochs)

# Hiển thị kết quả tốt nhất
print(f"Cấu hình tốt nhất: Loss = {best_config[0]}, Components = {best_config[1]}, Epochs = {best_config[2]}")
print(f"AUC cao nhất: {best_auc:.4f}")


Loss: warp, Components: 50, Epochs: 10, AUC: 0.5117
Loss: warp, Components: 50, Epochs: 30, AUC: 0.7930
Loss: warp, Components: 50, Epochs: 50, AUC: 0.8214
Loss: warp, Components: 100, Epochs: 10, AUC: 0.5354
Loss: warp, Components: 100, Epochs: 30, AUC: 0.8155
Loss: warp, Components: 100, Epochs: 50, AUC: 0.8219
Loss: warp, Components: 200, Epochs: 10, AUC: 0.5136
Loss: warp, Components: 200, Epochs: 30, AUC: 0.8156
Loss: warp, Components: 200, Epochs: 50, AUC: 0.8223
Loss: warp, Components: 300, Epochs: 10, AUC: 0.5186
Loss: warp, Components: 300, Epochs: 30, AUC: 0.8113
Loss: warp, Components: 300, Epochs: 50, AUC: 0.8226
Loss: bpr, Components: 50, Epochs: 10, AUC: 0.4988
Loss: bpr, Components: 50, Epochs: 30, AUC: 0.5053
Loss: bpr, Components: 50, Epochs: 50, AUC: 0.5235
Loss: bpr, Components: 100, Epochs: 10, AUC: 0.5040
Loss: bpr, Components: 100, Epochs: 30, AUC: 0.5029
Loss: bpr, Components: 100, Epochs: 50, AUC: 0.5386
Loss: bpr, Components: 200, Epochs: 10, AUC: 0.5024
Loss: 

Predict

In [14]:
def recommend_similar_items_with_scores(clicked_product, customer_id, model, dataset, user_features, item_features, data, top_n=10):
    """
    Gợi ý sản phẩm tương tự dựa trên Item, kèm điểm số.
    """
    # Lấy chỉ số ánh xạ của sản phẩm đã chọn
    item_index = dataset.mapping()[2][clicked_product]
    
    # Lấy chỉ số ánh xạ của khách hàng
    user_index = dataset.mapping()[0][customer_id]
    
    # Dự đoán điểm số cho tất cả sản phẩm dựa trên khách hàng và sản phẩm
    item_ids = np.arange(len(dataset.mapping()[2]))  # Danh sách tất cả sản phẩm
    scores = model.predict(
        user_ids=user_index,
        item_ids=item_ids,
        user_features=user_features,
        item_features=item_features
    )
    
    # Tìm sản phẩm tương tự sản phẩm đã chọn (embedding)
    _, item_embeddings = model.get_item_representations(item_features)
    similarities = np.dot(item_embeddings, item_embeddings[item_index])
    
    # Lọc và sắp xếp sản phẩm theo điểm số giảm dần
    item_mapping = {v: k for k, v in dataset.mapping()[2].items()}
    scored_items = [
        (item_mapping[i], similarities[i], scores[i]) 
        for i in np.argsort(-similarities) if item_mapping[i] != clicked_product
    ][:top_n]
    
    # Gắn thêm thông tin sản phẩm
    recommendations = [
        (item, data[data['Item Purchased'] == item]['Category'].values[0], similarity, score)
        for item, similarity, score in scored_items
    ]
    return recommendations

In [15]:
# Gợi ý sản phẩm dựa trên người dùng tương tự (User-based) kèm điểm số
def recommend_similar_users_with_scores(customer_id, model, dataset, user_features, item_features, data, top_n=10):
    """
    Gợi ý sản phẩm dựa trên người dùng tương tự, kèm điểm số.
    """
    # Lấy chỉ số ánh xạ của người dùng
    user_index = dataset.mapping()[0][customer_id]
    
    # Lấy embedding người dùng
    _, user_embeddings = model.get_user_representations(user_features)
    similarities = np.dot(user_embeddings, user_embeddings[user_index])
    
    # Tìm người dùng tương tự
    user_mapping = {v: k for k, v in dataset.mapping()[0].items()}
    top_users = [
        user_mapping[i] for i in np.argsort(-similarities) 
        if user_mapping[i] != customer_id
    ][:top_n]
    
    # Dự đoán điểm số cho tất cả sản phẩm
    item_ids = np.arange(len(dataset.mapping()[2]))
    scores = model.predict(
        user_ids=user_index,
        item_ids=item_ids,
        user_features=user_features,
        item_features=item_features
    ) 
    
    # Gợi ý sản phẩm từ những người dùng tương tự
    item_mapping = {v: k for k, v in dataset.mapping()[2].items()}
    recommendations = []
    for similar_user in top_users:
        similar_user_index = dataset.mapping()[0][similar_user]
        similar_scores = model.predict(
            user_ids=similar_user_index,
            item_ids=item_ids,
            user_features=user_features,
            item_features=item_features
        )
        recommended_items = [
            (item_mapping[i], scores[i]) for i in np.argsort(-similar_scores)
        ][:top_n]
        recommendations.extend(recommended_items)
    
    # Loại bỏ trùng lặp và sắp xếp theo điểm số
    recommendations = list(set(recommendations))
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]

In [17]:
def combined_recommendations(clicked_product, customer_id, model, dataset, user_features, item_features, data, top_n=10):
    """
    Gợi ý sản phẩm tổng hợp, tính trung bình giữa Item-based và User-based recommendations,
    sau đó sắp xếp theo giá trị trung bình (average score + similarity), đảm bảo giá trị dương.
    """
    # Gợi ý dựa trên sản phẩm (Item-based)
    item_recommendations = recommend_similar_items_with_scores(
        clicked_product, customer_id, model, dataset, user_features, item_features, data, top_n=top_n
    )
    
    # Chuyển danh sách Item-based thành dict {item: (category, similarity, score)}
    item_recommend_dict = {
        item: (data[data['Item Purchased'] == item]['Category'].values[0], similarity, score)
        for item, _, similarity, score in item_recommendations
    }
    
    # Gợi ý dựa trên người dùng tương tự (User-based)
    user_recommendations = recommend_similar_users_with_scores(
        customer_id, model, dataset, user_features, item_features, data, top_n=top_n
    )
    
    # Kết hợp gợi ý User-based vào dict
    for item, score in user_recommendations:
        category = data[data['Item Purchased'] == item]['Category'].values[0]
        if item in item_recommend_dict:
            # Tính trung bình similarity và score
            old_similarity = item_recommend_dict[item][1] if item_recommend_dict[item][1] is not None else 0
            old_score = item_recommend_dict[item][2]
            new_similarity = (old_similarity + 0) / 2 if old_similarity else None
            avg_score = (old_score + score) / 2
            item_recommend_dict[item] = (category, new_similarity, avg_score)
        else:
            # Thêm mục mới từ User-based
            item_recommend_dict[item] = (category, None, score)  # Similarity là None vì từ User-based
    
    # Dịch chuyển giá trị similarity và score để dương hết
    all_similarities = [sim for _, (_, sim, _) in item_recommend_dict.items() if sim is not None]
    all_scores = [score for _, (_, _, score) in item_recommend_dict.items()]
    
    min_similarity = min(all_similarities) if all_similarities else 0
    min_score = min(all_scores)
    
    for item in item_recommend_dict:
        category, similarity, score = item_recommend_dict[item]
        similarity = similarity - min_similarity if similarity is not None else None
        score = score - min_score
        item_recommend_dict[item] = (category, similarity, score)
    
    # Sắp xếp các sản phẩm theo score trung bình giảm dần
    sorted_recommendations = sorted(item_recommend_dict.items(), key=lambda x: x[1][2], reverse=True)
    
    # Chuyển thành danh sách kết quả
    final_recommendations = [
        (item, category, similarity, score)
        for item, (category, similarity, score) in sorted_recommendations[:top_n]
    ]
    
    return final_recommendations


In [18]:
# Nhập sản phẩm được nhấn và ID người dùng từ người dùng
clicked_product = input("Nhập sản phẩm mà bạn đã bấm (clicked product): ")
customer_id = input("Nhập ID người dùng (customer ID): ")

# Gợi ý tổng hợp
combined_recommendation_list = combined_recommendations(
    clicked_product, customer_id, model, dataset, user_features_matrix, item_features_matrix, data, top_n=50
)

# Hiển thị kết quả
print(f"\nSản phẩm '{clicked_product}' đã được nhấn.")
print("\nGợi ý sản phẩm (ưu tiên dựa trên sản phẩm, bổ sung từ người dùng tương tự nếu cần):")
for item, category, similarity, score in combined_recommendation_list:
    sim_text = f"Similarity: {similarity:.2f}" if similarity is not None else "Similarity: N/A"
    score_text = f"Score: {score:.2f}" if score is not None else "Score: N/A"
    print(f"  {item} (Category: {category}) - {sim_text} - {score_text}")



Sản phẩm 'Blouse' đã được nhấn.

Gợi ý sản phẩm (ưu tiên dựa trên sản phẩm, bổ sung từ người dùng tương tự nếu cần):
  Skirt (Category: Clothing) - Similarity: 16.36 - Score: 1.49
  Shirt (Category: Clothing) - Similarity: 16.32 - Score: 1.48
  Socks (Category: Clothing) - Similarity: 16.30 - Score: 1.48
  Pants (Category: Clothing) - Similarity: 16.28 - Score: 1.48
  Dress (Category: Clothing) - Similarity: 16.28 - Score: 1.48
  Sweater (Category: Clothing) - Similarity: 16.32 - Score: 1.48
  Hoodie (Category: Clothing) - Similarity: 16.29 - Score: 1.47
  Shorts (Category: Clothing) - Similarity: 16.29 - Score: 1.47
  Blouse (Category: Clothing) - Similarity: N/A - Score: 1.47
  T-shirt (Category: Clothing) - Similarity: 16.28 - Score: 1.47
  Jeans (Category: Clothing) - Similarity: 16.22 - Score: 1.46
  Shoes (Category: Footwear) - Similarity: 4.30 - Score: 0.27
  Sandals (Category: Footwear) - Similarity: 4.29 - Score: 0.26
  Sneakers (Category: Footwear) - Similarity: 4.27 - Score